# Imports

In [1]:
import pandas as pd
import re

# Macros

In [2]:
DATA_FOLDER = '../../data/'
FIG_FOLDER = '../../figures/'

In [3]:
3

3

# Utile Function

# Cleaning

## Inital Data import

In [4]:
df = pd.read_csv(DATA_FOLDER + 'loans_2007.csv')
OrgRowCount = df.shape[0]
OrgColCount = df.shape[1]

/Users/arnescheunemann/opt/anaconda3/envs/nf2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## dealing with NAN and numbers in strings 

In [5]:
# drop dublicated last rows seem to be sum of above and None lines
df = df.drop_duplicates()
## there seems to 3 with many empty cells so this is fine
df = df.dropna(thresh=50,axis=0)
df = df[~df.revol_util.isna()]
df = df[~df.last_credit_pull_d.isna()]

In [6]:
# transfer string 'num%' to number 0-1
df.revol_util = df.revol_util.apply(lambda x: str(x).replace('%','')).astype('float64')/100
df.int_rate = df.int_rate.apply(lambda x: str(x).replace('%','')).astype('float64')/100

In [7]:
# emp_length transfrom form string to number (<1 years ,  years, 10+ years etc)
def Trans_empValue(s):
    if type(s) is float:
        return 0.
    if re.search('\<',s):
        return 0.5
    else:
        return float(re.search('\d+',s).group())

df.emp_length = df.emp_length.apply(lambda x: Trans_empValue(x))

In [8]:
# no info about bakrupcie will be interpreted as none
df.pub_rec_bankruptcies = df.pub_rec_bankruptcies.fillna(0.0).astype('uint8')
# cap the very view that have amount of 2 ( could be spouses etc...)
df.pub_rec_bankruptcies = df.pub_rec_bankruptcies.apply(lambda x: 1 if x > 0 else 0)
df.pub_rec_bankruptcies.value_counts()

0    40476
1     1853
Name: pub_rec_bankruptcies, dtype: int64

In [9]:
# small feature engineering for EDA - actualy missplaced in the chapter but was easy to do
display(df[df.last_pymnt_d.isna()].loan_status.value_counts())
df['has_made_pymnt'] = df.last_pymnt_d.apply(lambda x: 1 if x else 0)
df = df.drop(['last_pymnt_d'],axis=1)

Charged Off                                            62
Does not meet the credit policy. Status:Charged Off    12
Name: loan_status, dtype: int64

## Droping columns with no information

In [10]:
# title could be filtered by keyword debt etc.. but catgoriy of purpose does better job thereby drop it
df = df.drop(['title'], axis=1)

In [11]:
# collections_12_mths_ex_med and chargeoff_within_12_mths drop the column all are the same = 0.0
df = df.drop(['chargeoff_within_12_mths', 'collections_12_mths_ex_med'], axis=1)
# application_type - only indvitual in the data can be droped
df = df.drop(['application_type'],axis=1)
# no info 
df = df.drop(['initial_list_status'],axis=1)
# no info in the data 
df = df.drop(['acc_now_delinq'],axis=1)
# not much info only 2 
df = df.drop(['delinq_amnt'],axis=1)
# only one value diffrent
df = df.drop(['tax_liens'],axis=1)
# all have no only one is yes
df = df.drop(['pymnt_plan'],axis=1)
# all are 1
df = df.drop(['policy_code'],axis=1)

## recheck on the NAN 

In [12]:
df.isnull().sum()

id                            0
member_id                     0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
sub_grade                     0
emp_title                  2590
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
loan_status                   0
purpose                       0
zip_code                      0
addr_state                    0
dti                           0
delinq_2yrs                   0
earliest_cr_line              0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
out_prncp                     0
out_prncp_inv                 0
total_py

--- > leavs only the emp_title -> push to feature engineering

## Dublication check on ids

In [13]:
print(df.id.duplicated().value_counts())
print(df.member_id.duplicated().value_counts())

False    42329
Name: id, dtype: int64
False    42329
Name: member_id, dtype: int64


--- > no doubles therfor the id has no value for prediction and nothing needs to be cleaned

In [14]:
df = df.drop(['id','member_id'],axis=1)

## Objects in the data check shellow if categorys

In [15]:
df.select_dtypes('object').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42329 entries, 0 to 42474
Data columns (total 13 columns):
term                   42329 non-null object
grade                  42329 non-null object
sub_grade              42329 non-null object
emp_title              39739 non-null object
home_ownership         42329 non-null object
verification_status    42329 non-null object
issue_d                42329 non-null object
loan_status            42329 non-null object
purpose                42329 non-null object
zip_code               42329 non-null object
addr_state             42329 non-null object
earliest_cr_line       42329 non-null object
last_credit_pull_d     42329 non-null object
dtypes: object(13)
memory usage: 4.5+ MB


In [16]:
## Term seems to be 36 month or 60 month ... keep category but make shorter and kill spaces
df.term = df.term.apply(lambda x: str(x).replace(' ', '').replace('months','m'))

In [17]:
#combine NONE and OTHER since there are only 2
# print(df.home_ownership.value_counts())
df.home_ownership = df.home_ownership.apply(lambda x: 'OTHER' if x=='NONE' else x)


_d columns have date info - > push to feature engineering dealing with datetime 

## Export Cleaned Data 

In [18]:
df.to_pickle(DATA_FOLDER + 'clean_pass_01p2.pkl')
print(f'{(1-df.shape[0]/OrgRowCount)*100}% data loss of cleaning')